
# Submetendo nosso modelo ao Kaggle

<p class="importante"> https://www.kaggle.com/c/titanic </p>

Podemos ver que no kaggle o dataset desse desafio esta separado em treino e teste! O teste é o dataset que iremos estimar os resultados!

<span class="atencao">Para realizarmos as predições é importante realizar as etapas de pre-processamento no dataset de teste! </span>

Obs.: nesse caso será dado o dataset de teste já pre-processado

In [1]:
import pandas
titanic_test = pandas.read_csv("https://databootcamp.nyc3.digitaloceanspaces.com/titanic_test.csv")

titanic_test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


### Nossa, mas vamos ter que processar tudo de novo?

Infelizmente sim... E precisa ser feito com os mesmos processos, senão pode dar erros no seu modelo que você não esperava!

In [4]:
titanic_test.isnull().sum()

PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64

Esse ainda tem uma pessoa que não sabemos o valor da passagem!

## Pipelines ao resgate!

Pipelines são uma forma de você sempre seguir um mesmo fluxo de processos. Quando a gente joga Mário a gente sempre sabe que ao entrar em um cano vamos sair do outro lado... O mesmo vale para nossos datasets.

![pipeline](https://databootcamp.nyc3.digitaloceanspaces.com/img/pipeline.jpg)

No **Sklearn** temos um fluxo de Pipeline para nos ajudar a aplicar sempre os mesmos processos nos dados.

#### Os Pipelines são super importantes para REPRODUTIBILIDADE

In [ ]:
from sklearn.pipeline import make_pipeline

Para realizar um passo do pipeline com uma função sua você precisa usar o **FunctionTransformer**. Ele permite que você use qualquer função em um pipeline, desde que ela receba um Dataframe e retorne um Dataframe

In [ ]:
from sklearn.preprocessing import FunctionTransformer, StandardScaler

Vamos fazer nossas funções de preprocessamento

In [ ]:
def add_age_filled(titanic):
    titanic["AgeFilled"] = titanic["Age"].fillna(titanic["Age"].median())
    return titanic

In [ ]:
def add_relatives(titanic):
    titanic["Relatives"] = titanic["SibSp"] + titanic["Parch"]
    return titanic

In [ ]:
def fill_fare(titanic):
    titanic["Fare"].fillna(titanic["Fare"].median(), inplace=True)
    return titanic

In [ ]:
def only_numbers(titanic):
    features = [
        "Pclass",
        "Sex",
        "SibSp",
        "Parch",
        "Relatives",
        "Embarked",
        "Fare",
        "AgeFilled"
    ]
    titanic_features = titanic[features]
    return pandas.get_dummies(titanic_features).drop("Sex_male", axis=1)


Vamos montar um Pipeline para todo nosso preprocessamento. Essas funções vão sendo executadas em um fluxo único

In [ ]:
preprocess_pipeline = make_pipeline(
    FunctionTransformer(add_age_filled),
    FunctionTransformer(add_relatives),
    FunctionTransformer(fill_fare),
    FunctionTransformer(only_numbers),
    StandardScaler()
)

Vamos aplicar nosso pipeline no Dataframe original do Titanic

In [ ]:
titanic = pandas.read_csv("https://databootcamp.nyc3.digitaloceanspaces.com/titanic.csv")
titanic.head()

Primeiro treinamos somente nosso Pipeline porque existem funções como o **StandardScaler** que precisam pegar os valores de média e desvio padrão com o dataset mais completo possível. O **fit** nos retorna um objeto Pipeline com todos os passos detalhados.

In [ ]:
preprocess_trained = preprocess_pipeline.fit(titanic)
preprocess_trained

Tendo o pipeline já treinado, podemos aplicar ele em qualquer novo conjunto de dados!

In [ ]:
titanic_preprocessed = preprocess_trained.transform(titanic)
titanic_preprocessed.shape

In [ ]:
label = titanic["Survived"]
label.shape

## <font color='blue'>Aplique nosso pipeline nos dados que vamos enviar para o Kaggle!</font>
![alt text](https://databootcamp.nyc3.digitaloceanspaces.com/img/atrasada-relogio-pulso-1116-1400x800.jpg)

In [ ]:
titanic_test = pandas.read_csv("https://databootcamp.nyc3.digitaloceanspaces.com/titanic_test.csv")

titanic_test.head()

In [ ]:
#solução

Agora que temos nossos dados processados vamos treinar nosso modelo nele!

In [ ]:
from sklearn.model_selection import train_test_split
x_treino, x_teste, y_treino, y_teste = train_test_split(
    titanic_preprocessed, label, test_size = 0.3, random_state=42
)

In [ ]:
from sklearn.linear_model import LogisticRegression

log_reg = LogisticRegression(random_state=42, solver="liblinear")
modelo = log_reg.fit(x_treino, y_treino)

## <font color='blue'>Agora consigam as predições e coloquem no formato do Kaggle</font>
Colunas necessárias:
- PassengerId
- Survived
![alt text](https://databootcamp.nyc3.digitaloceanspaces.com/img/atrasada-relogio-pulso-1116-1400x800.jpg)

In [ ]:
#solução

## Exportação do Modelo

É bem inconveniente precisar treinar o modelo sempre... Para isso, podemos exportar esse modelo para usarmos depois!

Para isso vamos precisar usar uma ferramenta chamada **joblib**.

In [ ]:
import joblib

Para salvar o modelo basta usar a função **dump**

In [ ]:
joblib.dump(modelo, "survived_model.joblib")

Para carregar novamente é tão simples quanto usar a função **load**

In [ ]:
novo_modelo = joblib.load("survived_model.joblib")
novo_modelo

Pode testar até em outro Notebook!